# Introduction

THis notebook contains he final notebook submission to the Kaggle competion predcit Calories challenge. I will start by first havoing code to the best possible solution based on the public leaderboard) and then explore other options. EDA and other stuff have been removed from this notebook for readability.

In [1]:
# imports

import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('PredictCalorie/train.csv')
test = pd.read_csv('PredictCalorie/test.csv')
sample = pd.read_csv('PredictCalorie/sample_submission.csv')

# LIghtGBM model

In [67]:
X = train.drop(columns=['id','Calories'])
y = train['Calories']

In [11]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import TargetEncoder
import lightgbm as lgb
import numpy as np

categorical_cols = ['Sex']
cv = KFold(n_splits=5, random_state=42, shuffle=True)  # Added n_splits for clarity
y_pred = np.zeros(len(test))

for idx_train, idx_valid in cv.split(X, y):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    X_test = test[X.columns].copy()

    encoder = TargetEncoder(random_state=42)

    # Reshape to 2D array for TargetEncoder
    X_train['Sex'] = encoder.fit_transform(X_train[['Sex']], y_train)[:, 0]  # Use [[:Sex]] and extract first column
    X_valid['Sex'] = encoder.transform(X_valid[['Sex']])[:, 0]
    X_test['Sex'] = encoder.transform(X_test[['Sex']])[:, 0]

    model = lgb.LGBMRegressor(
        n_iter=2000,
        max_depth=-1,
        num_leaves=1024,
        colsample_bytree=0.7,
        learning_rate=0.03,
        objective='l2',
        metric='rmse',
        verbosity=-1,
        max_bin=1024,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[
            lgb.log_evaluation(100),
            lgb.early_stopping(stopping_rounds=100)
        ],
    )

    y_pred += model.predict(X_test)



/var/folders/d3/s0jdmf113zs_4gc6g42z31fh0000gn/T/ipykernel_36927/1144295194.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Sex'] = encoder.fit_transform(X_train[['Sex']], y_train)[:, 0]  # Use [[:Sex]] and extract first column
/var/folders/d3/s0jdmf113zs_4gc6g42z31fh0000gn/T/ipykernel_36927/1144295194.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid['Sex'] = encoder.transform(X_valid[['Sex']])[:, 0]


Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 5.3963
[200]	valid_0's rmse: 3.66863


KeyboardInterrupt: 

In [ ]:
pred_lgbm = y_pred /5
sample_submission=pd.read_csv('sample_submission.csv')
submission_lgbm = pd.DataFrame({'id': sample_submission.id, 'Calories' : pred_lgbm})
submission_lgbm.to_csv('submission_lgbm.csv', index=False)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import TargetEncoder
import lightgbm as lgb
import numpy as np

categorical_cols = ['Sex']
cv = KFold(n_splits=5, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

# Log-transform the target
y_log = np.log1p(y)  # Apply log1p to the target

for idx_train, idx_valid in cv.split(X, y):
    X_train, y_train = X.iloc[idx_train], y_log.iloc[idx_train]  # Use log-transformed y
    X_valid, y_valid = X.iloc[idx_valid], y_log.iloc[idx_valid]
    X_test = test[X.columns].copy()

    encoder = TargetEncoder(random_state=42)
    X_train['Sex'] = encoder.fit_transform(X_train[['Sex']], y_train)[:, 0]
    X_valid['Sex'] = encoder.transform(X_valid[['Sex']])[:, 0]
    X_test['Sex'] = encoder.transform(X_test[['Sex']])[:, 0]

    model = lgb.LGBMRegressor(
        n_iter=2000,
        max_depth=-1,
        num_leaves=1024,
        colsample_bytree=0.7,
        learning_rate=0.03,
        objective='l2',
        metric='rmse',  # RMSE on log-transformed target = RMSLE
        verbosity=-1,
        max_bin=1024,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[
            lgb.log_evaluation(100),
            lgb.early_stopping(stopping_rounds=100)
        ],
    )

    # Exponentiate predictions to revert log-transform
    y_pred += np.expm1(model.predict(X_test))  # expm1 = exp(x) - 1

y_pred /= cv.get_n_splits()

pred_lgbm = y_pred
sample_submission=pd.read_csv('sample_submission.csv')
submission_lgbm = pd.DataFrame({'id': sample_submission.id, 'Calories' : pred_lgbm})
submission_lgbm.to_csv('submission_lgbm.csv', index=False)

# Use of CatBoost

In [12]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from math import sqrt

# Prepare training data
drop_cols = ['id', 'Calories']
X = train.drop(columns=drop_cols)
y = train['Calories']
cat_features = ['Sex']

# Split for evaluation
X_train,  y_train = X, y

# Define model
model = CatBoostRegressor(verbose=100, cat_features=cat_features)

# Define parameter grid (keep it small to start with)
param_grid = {
    'depth': [6, 8, 10],
    'learning_rate': [0.03, 0.07, 0.1],
    'l2_leaf_reg': [3,7],
    'iterations': [1000,2000]
}
param_grid = {
    'depth': [6, 8, 10],
    'learning_rate': [0.03, 0.07, 0.1],
    'l2_leaf_reg': [3, 7],
    'iterations': [1000, 2000],
    'bagging_temperature': [0, 0.5, 1],
    'random_strength': [1, 5, 10],
    'bootstrap_type': ['Bayesian', 'Bernoulli'],
    'grow_policy': ['SymmetricTree', 'Depthwise']
}
param_grid = {
    'depth': [6, 8, 10],
    'learning_rate': [0.03, 0.07, 0.1],
    'l2_leaf_reg': [3, 7],
    'iterations': [1000, 2000],
    'grow_policy': ['SymmetricTree', 'Depthwise']
}
'''
param_grid = {
    'depth': [6, 8],
    'learning_rate': [0.03, 0.1],
    'l2_leaf_reg': [3, 7],
    'iterations': [1000]
}
'''
# Grid search
grid_search = GridSearchCV(estimator=model,
                           param_grid=param_grid,
                           scoring='neg_root_mean_squared_error',
                           cv=3,
                           n_jobs=2)

grid_search.fit(X_train, y_train)

# Best results
print("✅ Best RMSE:", -grid_search.best_score_)
print("🎯 Best Parameters:", grid_search.best_params_)

best_model = grid_search.best_estimator_


KeyboardInterrupt: 

In [14]:
import itertools
import os
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Load / prepare training data
drop_cols = ['id', 'Calories']
X = train.drop(columns=drop_cols)
y = train['Calories']
cat_features = ['Sex']

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Parameter grid
param_grid = {
    'depth': [6, 8, 10],
    'learning_rate': [0.03, 0.07, 0.1],
    'l2_leaf_reg': [3, 7],
    'iterations': [1000, 2000],
    'random_strength': [1, 5, 10],
    'bootstrap_type': ['Bayesian', 'Bernoulli'],
    'grow_policy': ['SymmetricTree', 'Depthwise']
}

# Convert grid to list of dicts (one per combination)
def expand_grid(param_grid):
    keys = list(param_grid.keys())
    values = list(param_grid.values())
    combos = list(itertools.product(*values))
    return [dict(zip(keys, combo)) for combo in combos]

all_combinations = expand_grid(param_grid)

# File to cache results
results_file = "catboost_grid_results_calorie.csv"
if os.path.exists(results_file):
    results_df = pd.read_csv(results_file)
else:
    results_df = pd.DataFrame(columns=list(param_grid.keys()) + ["rmse"])

# Run only for new combinations
new_results = []

for params in all_combinations:
    # Check if this combination has already been run
    mask = (results_df[list(params)] == pd.Series(params)).all(axis=1)
    if mask.any():
        print(f"✅ Skipping already-run: {params}")
        continue

    print(f"🚀 Running: {params}")
    try:
        model = CatBoostRegressor(cat_features=cat_features, verbose=0, **params)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        rmse = sqrt(mean_squared_error(y_val, preds))
        print(f"→ RMSE: {rmse:.4f}")

        # Save result after each run
        result_row = {**params, 'rmse': rmse}
        results_df = pd.concat([results_df, pd.DataFrame([result_row])], ignore_index=True)
        results_df.to_csv(results_file, index=False)
        print(f"💾 Result saved to {results_file}")

    except Exception as e:
        print(f"❌ Error with params {params}: {e}")

# Append new results and save
if new_results:
    results_df = pd.concat([results_df, pd.DataFrame(new_results)], ignore_index=True)
    results_df.to_csv(results_file, index=False)
    print(f"💾 Results saved to {results_file}")

# Show best
best = results_df.sort_values('rmse').iloc[0]
print("\n🏆 Best Config:")
print(best)


🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 1, 'bootstrap_type': 'Bayesian', 'grow_policy': 'SymmetricTree'}
→ RMSE: 3.5967
💾 Result saved to catboost_grid_results_calorie.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 1, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Depthwise'}


/var/folders/d3/s0jdmf113zs_4gc6g42z31fh0000gn/T/ipykernel_36927/4010438909.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([result_row])], ignore_index=True)


→ RMSE: 3.6077
💾 Result saved to catboost_grid_results_calorie.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 1, 'bootstrap_type': 'Bernoulli', 'grow_policy': 'SymmetricTree'}
→ RMSE: 3.5944
💾 Result saved to catboost_grid_results_calorie.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 1, 'bootstrap_type': 'Bernoulli', 'grow_policy': 'Depthwise'}
→ RMSE: 3.5971
💾 Result saved to catboost_grid_results_calorie.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 5, 'bootstrap_type': 'Bayesian', 'grow_policy': 'SymmetricTree'}
→ RMSE: 3.6030
💾 Result saved to catboost_grid_results_calorie.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 5, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Depthwise'}
→ RMSE: 3.6149
💾 Result saved to catboost_grid_results_calorie

In [16]:
# Define columns to exclude from features
exclude_cols = ['id', 'Calories']
target_col = 'Calories'

# Split features and target from train
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Prepare test set (same feature columns)
X_test = test.drop(columns=['id'])

# Identify categorical features (assumes object types are categorical)
categorical_features = ['Sex']

# Optional: split validation set from train (for early stopping)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define Pools
train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
val_pool = Pool(X_val, y_val, cat_features=categorical_features)
test_pool = Pool(X_test, cat_features=categorical_features)

# Train the model
model = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.03,
    depth=10,
    random_strength=5,
    l2_leaf_reg=7,
    bootstrap_type='Bernoulli',
    grow_policy='SymmetricTree',
    eval_metric='RMSE',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=100
)

model.fit(train_pool, eval_set=val_pool)

# Predict on test set
test_preds = model.predict(test_pool)

# Add predictions to test dataframe
test['Calories'] = test_preds

# Optional: preview results
print(test[['id','Calories']].head())

0:	learn: 60.6811318	test: 60.5082800	best: 60.5082800 (0)	total: 36.1ms	remaining: 1m 12s
100:	learn: 5.7573990	test: 5.7426415	best: 5.7426415 (100)	total: 1.49s	remaining: 28.1s
200:	learn: 3.8117644	test: 3.8401566	best: 3.8401566 (200)	total: 3.02s	remaining: 27s
300:	learn: 3.6295186	test: 3.6791427	best: 3.6791427 (300)	total: 4.77s	remaining: 26.9s
400:	learn: 3.5529231	test: 3.6217363	best: 3.6217363 (400)	total: 6.49s	remaining: 25.9s
500:	learn: 3.5078001	test: 3.5929045	best: 3.5929045 (500)	total: 8.19s	remaining: 24.5s
600:	learn: 3.4760466	test: 3.5780044	best: 3.5780044 (600)	total: 9.86s	remaining: 23s
700:	learn: 3.4526859	test: 3.5672550	best: 3.5672550 (700)	total: 11.6s	remaining: 21.5s
800:	learn: 3.4337295	test: 3.5609999	best: 3.5609999 (800)	total: 13.3s	remaining: 19.9s
900:	learn: 3.4170465	test: 3.5564010	best: 3.5564010 (900)	total: 15s	remaining: 18.3s
1000:	learn: 3.4020588	test: 3.5525224	best: 3.5525209 (999)	total: 16.8s	remaining: 16.8s
1100:	learn: 3

In [19]:
test[['id','Calories']].to_csv('catboost.csv',index=False)

In [68]:
import numpy as np
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

# Define columns to exclude from features
exclude_cols = ['id', 'Calories']
target_col = 'Calories'

# Split features and target from train
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Log-transform the target for RMSLE
y_train_log = np.log1p(y_train)

# Prepare test set (same feature columns)
X_test = test.drop(columns=['id'])

# Identify categorical features
categorical_features = ['Sex']

# Initialize KFold
cv = KFold(n_splits=10, random_state=42, shuffle=True)

# Initialize array to store test predictions
test_preds = np.zeros(len(test))

# K-Fold Cross-Validation
for fold, (idx_train, idx_valid) in enumerate(cv.split(X_train, y_train)):
    print(f'\nFold {fold + 1}')
    
    # Split data
    X_tr, X_val = X_train.iloc[idx_train], X_train.iloc[idx_valid]
    y_tr, y_val = y_train_log.iloc[idx_train], y_train_log.iloc[idx_valid]
    
    # Define Pools
    train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
    val_pool = Pool(X_val, y_val, cat_features=categorical_features)
    test_pool = Pool(X_test, cat_features=categorical_features)
    
    # Train the model
    model = CatBoostRegressor(
        iterations=2000,
        learning_rate=0.03,
        depth=10,
        random_strength=5,
        l2_leaf_reg=7,
        bootstrap_type='Bernoulli',
        grow_policy='SymmetricTree',
        eval_metric='RMSE',  # RMSE on log-transformed target = RMSLE
        random_seed=42,
        early_stopping_rounds=50,
        verbose=100
    )
    
    model.fit(train_pool, eval_set=val_pool)
    
    # Predict on test set and revert log-transform
    fold_preds = np.expm1(model.predict(test_pool))
    test_preds += fold_preds

# Average predictions across folds
test_preds /= cv.get_n_splits()

# Add predictions to test dataframe
test['Calories'] = test_preds

# Optional: preview results
print(test[['id', 'Calories']].head())


Fold 1
0:	learn: 0.9359762	test: 0.9373258	best: 0.9373258 (0)	total: 43.7ms	remaining: 1m 27s
100:	learn: 0.0913315	test: 0.0922263	best: 0.0922263 (100)	total: 1.88s	remaining: 35.4s
200:	learn: 0.0630400	test: 0.0640533	best: 0.0640533 (200)	total: 3.78s	remaining: 33.9s
300:	learn: 0.0603645	test: 0.0616511	best: 0.0616511 (300)	total: 5.75s	remaining: 32.5s
400:	learn: 0.0594146	test: 0.0609400	best: 0.0609400 (400)	total: 7.67s	remaining: 30.6s
500:	learn: 0.0587948	test: 0.0605739	best: 0.0605739 (500)	total: 9.71s	remaining: 29s
600:	learn: 0.0583370	test: 0.0603406	best: 0.0603406 (600)	total: 11.7s	remaining: 27.2s
700:	learn: 0.0579921	test: 0.0602328	best: 0.0602328 (700)	total: 13.7s	remaining: 25.4s
800:	learn: 0.0577183	test: 0.0601513	best: 0.0601513 (800)	total: 15.8s	remaining: 23.6s
900:	learn: 0.0574745	test: 0.0600854	best: 0.0600854 (900)	total: 17.8s	remaining: 21.7s
1000:	learn: 0.0572478	test: 0.0600381	best: 0.0600381 (1000)	total: 19.7s	remaining: 19.7s
1100

In [69]:
test[['id','Calories']].to_csv('catboost_10_fold.csv',index=False)

In [22]:
lgbm = pd.read_csv('submission_lgbm (2).csv')

In [23]:
catboost = pd.read_csv('catboost_k_fold.csv')

In [74]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np

# Bin 'Duration' into quantile-based categories (e.g., 5 groups)
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Add Duration binning for stratification
duration_bins = pd.qcut(train['Duration'], q=10, labels=False)  # or use pd.cut for fixed bins

# Initialize StratifiedKFold using duration bins
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

test_preds = np.zeros(len(test))


# Proceed with the same logic — just replace KFold with StratifiedKFold
for fold, (idx_train, idx_valid) in enumerate(cv.split(X_train, duration_bins)):
    print(f'\nFold {fold + 1}')
    
    X_tr, X_val = X_train.iloc[idx_train], X_train.iloc[idx_valid]
    y_tr, y_val = np.log1p(y_train.iloc[idx_train]), np.log1p(y_train.iloc[idx_valid])
    
    # CatBoost Pools
    train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
    val_pool = Pool(X_val, y_val, cat_features=categorical_features)
    test_pool = Pool(X_test, cat_features=categorical_features)
    
    # Train and predict as before...
    model = CatBoostRegressor(
        iterations=2000,
        learning_rate=0.03,
        depth=10,
        random_strength=5,
        l2_leaf_reg=7,
        bootstrap_type='Bernoulli',
        grow_policy='SymmetricTree',
        eval_metric='RMSE',
        random_seed=42,
        early_stopping_rounds=50,
        verbose=100
    )
    
    model.fit(train_pool, eval_set=val_pool)
    test_preds += np.expm1(model.predict(test_pool))

test_preds /= cv.get_n_splits()
test['Calories'] = test_preds
test[['id','Calories']].to_csv('catboost_5_fold_str.csv',index=False)


Fold 1
0:	learn: 0.9366100	test: 0.9354519	best: 0.9354519 (0)	total: 33.6ms	remaining: 1m 7s
100:	learn: 0.0924376	test: 0.0926483	best: 0.0926483 (100)	total: 1.57s	remaining: 29.5s
200:	learn: 0.0637060	test: 0.0646417	best: 0.0646417 (200)	total: 3.27s	remaining: 29.2s
300:	learn: 0.0606240	test: 0.0618747	best: 0.0618747 (300)	total: 5.35s	remaining: 30.2s
400:	learn: 0.0595081	test: 0.0610308	best: 0.0610305 (399)	total: 7.21s	remaining: 28.7s
500:	learn: 0.0588160	test: 0.0605886	best: 0.0605886 (500)	total: 9.31s	remaining: 27.8s
600:	learn: 0.0583254	test: 0.0603792	best: 0.0603782 (599)	total: 11s	remaining: 25.7s
700:	learn: 0.0579711	test: 0.0602938	best: 0.0602938 (700)	total: 12.8s	remaining: 23.7s
800:	learn: 0.0576959	test: 0.0602223	best: 0.0602223 (800)	total: 14.7s	remaining: 22s
900:	learn: 0.0574552	test: 0.0601871	best: 0.0601871 (900)	total: 16.4s	remaining: 20s
1000:	learn: 0.0572292	test: 0.0601481	best: 0.0601469 (991)	total: 18.4s	remaining: 18.4s
1100:	lear

In [76]:
test.max()

id                999999
Sex                 male
Age                   79
Height             219.0
Weight             126.0
Duration            30.0
Heart_Rate         128.0
Body_Temp           41.5
Calories      298.003578
dtype: object

In [27]:

# Merge the two dataframes on 'id'
merged = pd.merge(lgbm[['id', 'Calories']], catboost[['id', 'Calories']],
                  on='id', suffixes=('_lgbm', '_catboost'))

# Calculate the average of the two calorie columns
merged['Calories'] = merged[['Calories_lgbm', 'Calories_catboost']].mean(axis=1)

# Result: id and average calories
result = merged[['id', 'Calories']]
result.to_csv('ensemble_catboost_lgbm.csv',index=False)

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


train = pd.read_csv('PredictCalorie/train.csv')
test = pd.read_csv('PredictCalorie/test.csv')

# Define columns to exclude from features
exclude_cols = ['id', 'Calories']
target_col = 'Calories'

# Convert 'Sex' to binary (1 for Male, 0 for Female)
train['Sex'] = train['Sex'].map({'Male': 1, 'Female': 0})
test['Sex'] = test['Sex'].map({'Male': 1, 'Female': 0})

# Split features and target from train
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Log-transform the target for RMSLE
y_train_log = np.log1p(y_train)

# Prepare test set (same feature columns)
X_test = test.drop(columns=['id'])

# Check for non-numeric columns
print("Data types in X_train:\n", X_train.dtypes)
print("Data types in X_test:\n", X_test.dtypes)

# Check for infinite values
print("Infinite values in X_train:", np.any(np.isinf(X_train)))
print("Infinite values in X_test:", np.any(np.isinf(X_test)))

# Check for zero-variance features
std_dev = X_train.std()
zero_variance_cols = std_dev[std_dev == 0].index
print("Zero-variance columns:", zero_variance_cols.tolist())

# Remove zero-variance columns (if any)
if len(zero_variance_cols) > 0:
    print(f"Removing zero-variance columns: {zero_variance_cols.tolist()}")
    X_train = X_train.drop(columns=zero_variance_cols)
    X_test = X_test.drop(columns=zero_variance_cols)

# Initialize StandardScaler
scaler = StandardScaler()

# Initialize KFold
cv = KFold(n_splits=5, random_state=42, shuffle=True)

# Initialize array to store test predictions
test_preds = np.zeros(len(test))

# Define hyperparameter grid for Ridge regression
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
    'fit_intercept': [True, False]
}

# K-Fold Cross-Validation
for fold, (idx_train, idx_valid) in enumerate(cv.split(X_train, y_train)):
    print(f'\nFold {fold + 1}')
    
    # Split data
    X_tr, X_val = X_train.iloc[idx_train], X_train.iloc[idx_valid]
    y_tr, y_val = y_train_log.iloc[idx_train], y_train_log.iloc[idx_valid]
    
    # Scale features, with handling for zero-variance
    try:
        X_tr_scaled = scaler.fit_transform(X_tr)
        X_val_scaled = scaler.transform(X_val)
        X_test_scaled = scaler.transform(X_test)
        
        # Check for NaN after scaling
        if np.any(np.isnan(X_tr_scaled)):
            print(f"NaN values introduced in X_tr_scaled for fold {fold + 1}")
            break
        if np.any(np.isnan(X_val_scaled)):
            print(f"NaN values introduced in X_val_scaled for fold {fold + 1}")
            break
        if np.any(np.isnan(X_test_scaled)):
            print(f"NaN values introduced in X_test_scaled for fold {fold + 1}")
            break
    except Exception as e:
        print(f"Scaling error in fold {fold + 1}: {e}")
        break
    
    # Initialize Ridge regression model
    model = Ridge(random_state=42)
    
    # Perform GridSearchCV for hyperparameter optimization
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',  # Optimize for MSE on log-transformed target
        cv=3,
        n_jobs=-1,
        verbose=1
    )
    
    # Fit GridSearchCV
    try:
        grid_search.fit(X_tr_scaled, y_tr)
    except Exception as e:
        print(f"Error in GridSearchCV for fold {fold + 1}: {e}")
        continue
    
    # Best model from grid search
    best_model = grid_search.best_estimator_
    print(f"Best parameters for fold {fold + 1}: {grid_search.best_params_}")
    
    # Predict on validation set and compute RMSLE
    val_preds_log = best_model.predict(X_val_scaled)
    val_preds = np.expm1(val_preds_log)  # Revert log-transform
    y_val_orig = np.expm1(y_val)
    rmsle = np.sqrt(mean_squared_error(np.log1p(y_val_orig), np.log1p(val_preds)))
    print(f"Fold {fold + 1} RMSLE: {rmsle:.4f}")
    
    # Predict on test set
    fold_preds = np.expm1(best_model.predict(X_test_scaled))
    test_preds += fold_preds

# Average predictions across folds
test_preds /= cv.get_n_splits()

# Add predictions to test dataframe
test['Calories'] = test_preds

# Optional: preview results
print(test[['id', 'Calories']].head())

Data types in X_train:
 Sex           float64
Age             int64
Height        float64
Weight        float64
Duration      float64
Heart_Rate    float64
Body_Temp     float64
dtype: object
Data types in X_test:
 Sex           float64
Age             int64
Height        float64
Weight        float64
Duration      float64
Heart_Rate    float64
Body_Temp     float64
dtype: object
Infinite values in X_train: False
Infinite values in X_test: False
Zero-variance columns: []

Fold 1
NaN values introduced in X_tr_scaled for fold 1
       id  Calories
0  750000       0.0
1  750001       0.0
2  750002       0.0
3  750003       0.0
4  750004       0.0


/Users/ssilva/miniconda3/lib/python3.11/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/ssilva/miniconda3/lib/python3.11/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/ssilva/miniconda3/lib/python3.11/site-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [37]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Define columns to exclude from features
exclude_cols = ['id', 'Calories']
target_col = 'Calories'

# Convert 'Sex' to binary (1 for Male, 0 for Female)
train['Sex'] = train['Sex'].map({'Male': 1, 'Female': 0})
test['Sex'] = test['Sex'].map({'Male': 1, 'Female': 0})

# Split features and target from train
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Log-transform the target for RMSLE
y_train_log = np.log1p(y_train)

# Prepare test set (drop both 'id' and 'Calories')
X_test = test.drop(columns=['id', 'Calories'], errors='ignore')

# Check for non-numeric columns and column consistency
print("Data types in X_train:\n", X_train.dtypes)
print("Data types in X_test:\n", X_test.dtypes)
print("Columns in X_train:", X_train.columns.tolist())
print("Columns in X_test:", X_test.columns.tolist())

# Check for infinite values
print("Infinite values in X_train:", np.any(np.isinf(X_train)))
print("Infinite values in X_test:", np.any(np.isinf(X_test)))

# Check for zero-variance features in entire X_train
std_dev = X_train.std()
zero_variance_cols = std_dev[std_dev == 0].index
print("Zero-variance columns in X_train:", zero_variance_cols.tolist())

# Remove zero-variance columns (if any)
if len(zero_variance_cols) > 0:
    print(f"Removing zero-variance columns: {zero_variance_cols.tolist()}")
    X_train = X_train.drop(columns=exclude_cols)

# Initialize StandardScaler
scaler = StandardScaler()

# Initialize KFold
cv = KFold(n_splits=5, random_state=42, shuffle=True)

# Initialize array to store test predictions
test_preds = np.zeros(len(test))

# Define hyperparameter grid for Ridge regression
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
    'fit_intercept': [True, False]
}

# K-Fold Cross-Validation
for fold, (idx_train, idx_valid) in enumerate(cv.split(X_train, y_train)):
    print(f'\nFold {fold + 1}')
    
    # Split data
    X_tr, X_val = X_train.iloc[idx_train], X_train.iloc[idx_valid]
    y_tr, y_val = y_train_log.iloc[idx_train], y_train_log.iloc[idx_valid]
    
    # Check for zero-variance features in this fold's training split
    std_dev_fold = X_tr.std()
    zero_variance_cols_fold = std_dev_fold[std_dev_fold == 0].index
    print(f"Zero-variance columns in Fold {fold + 1} training split:", zero_variance_cols_fold.tolist())
    
    # Remove zero-variance columns for this fold
    X_tr_fold = X_tr.drop(columns=zero_variance_cols_fold, errors='ignore')
    X_val_fold = X_val.drop(columns=zero_variance_cols_fold, errors='ignore')
    X_test_fold = X_test.drop(columns=zero_variance_cols_fold, errors='ignore')
    
    # Store column names before scaling (for debugging)
    feature_names = X_tr_fold.columns
    
    # Scale features
    try:
        X_tr_scaled = scaler.fit_transform(X_tr_fold)
        X_val_scaled = scaler.transform(X_val_fold)
        X_test_scaled = scaler.transform(X_test_fold)
        
        # Check for NaN after scaling
        if np.any(np.isnan(X_tr_scaled)):
            nan_indices = np.any(np.isnan(X_tr_scaled), axis=0)
            nan_cols = feature_names[nan_indices].tolist()
            print(f"NaN values introduced in X_tr_scaled for Fold {fold + 1} in columns: {nan_cols}")
            print(f"Standard deviations of problematic columns in X_tr_fold:\n{X_tr_fold[nan_cols].std()}")
            break
        if np.any(np.isnan(X_val_scaled)):
            nan_indices = np.any(np.isnan(X_val_scaled), axis=0)
            nan_cols = feature_names[nan_indices].tolist()
            print(f"NaN values introduced in X_val_scaled for Fold {fold + 1} in columns: {nan_cols}")
            print(f"Standard deviations of problematic columns in X_tr_fold:\n{X_tr_fold[nan_cols].std()}")
            break
        if np.any(np.isnan(X_test_scaled)):
            nan_indices = np.any(np.isnan(X_test_scaled), axis=0)
            nan_cols = feature_names[nan_indices].tolist()
            print(f"NaN values introduced in X_test_scaled for Fold {fold + 1} in columns: {nan_cols}")
            print(f"Standard deviations of problematic columns in X_tr_fold:\n{X_tr_fold[nan_cols].std()}")
            break
    except Exception as e:
        print(f"Scaling error in Fold {fold + 1}: {e}")
        break
    
    # Initialize Ridge regression model
    model = Ridge(random_state=42)
    
    # Perform GridSearchCV for hyperparameter optimization
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',  # Optimize for MSE on log-transformed target
        cv=3,
        n_jobs=-1,
        verbose=1
    )
    
    # Fit GridSearchCV
    try:
        grid_search.fit(X_tr_scaled, y_tr)
    except Exception as e:
        print(f"Error in GridSearchCV for Fold {fold + 1}: {e}")
        continue
    
    # Best model from grid search
    best_model = grid_search.best_estimator_
    print(f"Best parameters for Fold {fold + 1}: {grid_search.best_params_}")
    
    # Predict on validation set and compute RMSLE
    val_preds_log = best_model.predict(X_val_scaled)
    val_preds = np.expm1(val_preds_log)  # Revert log-transform
    y_val_orig = np.expm1(y_val)
    rmsle = np.sqrt(mean_squared_error(np.log1p(y_val_orig), np.log1p(val_preds)))
    print(f"Fold {fold + 1} RMSLE: {rmsle:.4f}")
    
    # Predict on test set
    fold_preds = np.expm1(best_model.predict(X_test_scaled))
    test_preds += fold_preds

# Average predictions across folds
test_preds /= cv.get_n_splits()

# Add predictions to test dataframe
test['Calories'] = test_preds

# Optional: preview results
print(test[['id', 'Calories']].head())

Data types in X_train:
 Sex           float64
Age             int64
Height        float64
Weight        float64
Duration      float64
Heart_Rate    float64
Body_Temp     float64
dtype: object
Data types in X_test:
 Sex           float64
Age             int64
Height        float64
Weight        float64
Duration      float64
Heart_Rate    float64
Body_Temp     float64
dtype: object
Columns in X_train: ['Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
Columns in X_test: ['Sex', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
Infinite values in X_train: False
Infinite values in X_test: False
Zero-variance columns in X_train: []

Fold 1
Zero-variance columns in Fold 1 training split: []
NaN values introduced in X_tr_scaled for Fold 1 in columns: ['Sex']
Standard deviations of problematic columns in X_tr_fold:
Sex   NaN
dtype: float64
       id  Calories
0  750000       0.0
1  750001       0.0
2  750002       0.0
3  750003       0.0
4  750004       0

/Users/ssilva/miniconda3/lib/python3.11/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/ssilva/miniconda3/lib/python3.11/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/ssilva/miniconda3/lib/python3.11/site-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [41]:
import numpy as np
import pandas as pd
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

# Target and feature prep
exclude_cols = ['id', 'Calories']
target_col = 'Calories'

X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]
y_train_log = np.log1p(y_train)

X_test = test.drop(columns=['id'])

# Categorical preprocessing
categorical_features = ['Sex']
numerical_features = [col for col in X_train.columns if col not in categorical_features]

from sklearn.preprocessing import StandardScaler

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), categorical_features),
        ('num', StandardScaler(), numerical_features)
    ]
)

# KFold setup
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize test prediction accumulator
test_preds = np.zeros(len(test))

# Alpha values to try for Ridge
alphas = np.logspace(-3, 3, 50)

# Cross-validation loop
for fold, (idx_train, idx_valid) in enumerate(cv.split(X_train, y_train_log)):
    print(f'\nFold {fold + 1}')
    
    X_tr, X_val = X_train.iloc[idx_train], X_train.iloc[idx_valid]
    y_tr, y_val = y_train_log.iloc[idx_train], y_train_log.iloc[idx_valid]
    
    # RidgeCV automatically selects the best alpha using internal CV
    model = make_pipeline(
        preprocessor,
        RidgeCV(alphas=alphas, scoring='neg_mean_squared_error', cv=5)
    )
    
    model.fit(X_tr, y_tr)
    
    fold_preds = np.expm1(model.predict(X_test))
    test_preds += fold_preds

# Average across folds
test_preds /= cv.get_n_splits()

# Store predictions in test dataframe
test['Calories'] = test_preds

# Preview
print(test[['id', 'Calories']].head())


Fold 1

Fold 2

Fold 3

Fold 4

Fold 5
       id    Calories
0  750000   34.314976
1  750001   97.971926
2  750002   86.705360
3  750003  135.314130
4  750004   77.979985


In [42]:
test[['id', 'Calories']].to_csv('linear_reg_1.csv',index=False)

# Add more features

In [43]:
train = pd.read_csv('PredictCalorie/train.csv')
test = pd.read_csv('PredictCalorie/test.csv')
sample = pd.read_csv('PredictCalorie/sample_submission.csv')
train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [44]:
train['bmi']=train['Weight']/(train['Height']*train['Height'])*100*100
test['bmi']=test['Weight']/(test['Height']*test['Height'])*100*100

train['dur_body_heart']=train['Duration']*train['Heart_Rate']*train['Body_Temp']
test['dur_body_heart']=test['Duration']*test['Heart_Rate']*test['Body_Temp']



In [46]:
import numpy as np
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

# Define columns to exclude from features
exclude_cols = ['id', 'Calories']
target_col = 'Calories'

# Split features and target from train
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Log-transform the target for RMSLE
y_train_log = np.log1p(y_train)

# Prepare test set (same feature columns)
X_test = test.drop(columns=['id'])

# Identify categorical features
categorical_features = ['Sex']

# Initialize KFold
cv = KFold(n_splits=5, random_state=42, shuffle=True)

# Initialize array to store test predictions
test_preds = np.zeros(len(test))

# K-Fold Cross-Validation
for fold, (idx_train, idx_valid) in enumerate(cv.split(X_train, y_train)):
    print(f'\nFold {fold + 1}')
    
    # Split data
    X_tr, X_val = X_train.iloc[idx_train], X_train.iloc[idx_valid]
    y_tr, y_val = y_train_log.iloc[idx_train], y_train_log.iloc[idx_valid]
    
    # Define Pools
    train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
    val_pool = Pool(X_val, y_val, cat_features=categorical_features)
    test_pool = Pool(X_test, cat_features=categorical_features)
    
    # Train the model
    model = CatBoostRegressor(
        iterations=5000,
        learning_rate=0.03,
        depth=10,
        random_strength=5,
        l2_leaf_reg=7,
        bootstrap_type='Bernoulli',
        grow_policy='SymmetricTree',
        eval_metric='RMSE',  # RMSE on log-transformed target = RMSLE
        random_seed=42,
        early_stopping_rounds=100,
        verbose=100
    )
    
    model.fit(train_pool, eval_set=val_pool)
    
    # Predict on test set and revert log-transform
    fold_preds = np.expm1(model.predict(test_pool))
    test_preds += fold_preds

# Average predictions across folds
test_preds /= cv.get_n_splits()

# Add predictions to test dataframe
test['Calories'] = test_preds

# Optional: preview results
test[['id','Calories']].to_csv('catboost_k_fold_new_2.csv',index=False)
print(test[['id', 'Calories']].head())


Fold 1
0:	learn: 0.9362856	test: 0.9354721	best: 0.9354721 (0)	total: 55.4ms	remaining: 4m 37s
100:	learn: 0.0897575	test: 0.0894931	best: 0.0894931 (100)	total: 1.74s	remaining: 1m 24s
200:	learn: 0.0628148	test: 0.0629522	best: 0.0629522 (200)	total: 3.43s	remaining: 1m 21s
300:	learn: 0.0607417	test: 0.0612467	best: 0.0612467 (300)	total: 5.39s	remaining: 1m 24s
400:	learn: 0.0598463	test: 0.0606019	best: 0.0606019 (400)	total: 7.13s	remaining: 1m 21s
500:	learn: 0.0592088	test: 0.0602172	best: 0.0602172 (500)	total: 8.9s	remaining: 1m 19s
600:	learn: 0.0586550	test: 0.0599508	best: 0.0599508 (600)	total: 10.7s	remaining: 1m 18s
700:	learn: 0.0582568	test: 0.0597951	best: 0.0597951 (700)	total: 12.5s	remaining: 1m 16s
800:	learn: 0.0579345	test: 0.0596987	best: 0.0596987 (800)	total: 14.4s	remaining: 1m 15s
900:	learn: 0.0576552	test: 0.0596253	best: 0.0596236 (899)	total: 16.1s	remaining: 1m 13s
1000:	learn: 0.0574121	test: 0.0595825	best: 0.0595825 (1000)	total: 17.9s	remaining: 

In [48]:
def calculate_bmr(row):
    if row['Sex'].lower() == 'male':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] + 5
    elif row['Sex'].lower() == 'female':
        return 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] - 161
    else:
        return np.nan  # handle unknown or missing Sex

train['BMR'] = train.apply(calculate_bmr, axis=1)
test['BMR'] = test.apply(calculate_bmr, axis=1)

In [49]:
train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,bmi,dur_body_heart,BMR
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0,22.955684,107666.0,1826.25
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0,22.582709,26996.0,1137.75
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0,24.690405,23402.4,1230.25
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0,24.414062,106837.5,2005.00
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0,22.136740,103530.0,1296.50


In [50]:
import numpy as np
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

# Define columns to exclude from features
exclude_cols = ['id', 'Calories']
target_col = 'Calories'

# Split features and target from train
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Log-transform the target for RMSLE
y_train_log = np.log1p(y_train)

# Prepare test set (same feature columns)
X_test = test.drop(columns=['id'])

# Identify categorical features
categorical_features = ['Sex']

# Initialize KFold
cv = KFold(n_splits=5, random_state=42, shuffle=True)

# Initialize array to store test predictions
test_preds = np.zeros(len(test))

# K-Fold Cross-Validation
for fold, (idx_train, idx_valid) in enumerate(cv.split(X_train, y_train)):
    print(f'\nFold {fold + 1}')
    
    # Split data
    X_tr, X_val = X_train.iloc[idx_train], X_train.iloc[idx_valid]
    y_tr, y_val = y_train_log.iloc[idx_train], y_train_log.iloc[idx_valid]
    
    # Define Pools
    train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
    val_pool = Pool(X_val, y_val, cat_features=categorical_features)
    test_pool = Pool(X_test, cat_features=categorical_features)
    
    # Train the model
    model = CatBoostRegressor(
        iterations=5000,
        learning_rate=0.03,
        depth=10,
        random_strength=5,
        l2_leaf_reg=7,
        bootstrap_type='Bernoulli',
        grow_policy='SymmetricTree',
        eval_metric='RMSE',  # RMSE on log-transformed target = RMSLE
        random_seed=42,
        early_stopping_rounds=100,
        verbose=100
    )
    
    model.fit(train_pool, eval_set=val_pool)
    
    # Predict on test set and revert log-transform
    fold_preds = np.expm1(model.predict(test_pool))
    test_preds += fold_preds

# Average predictions across folds
test_preds /= cv.get_n_splits()

# Add predictions to test dataframe
test['Calories'] = test_preds

# Optional: preview results
test[['id','Calories']].to_csv('catboost_k_fold_new_2.csv',index=False)
print(test[['id', 'Calories']].head())


Fold 1
0:	learn: 0.9366597	test: 0.9357933	best: 0.9357933 (0)	total: 36ms	remaining: 2m 59s
100:	learn: 0.0897700	test: 0.0893913	best: 0.0893913 (100)	total: 1.83s	remaining: 1m 29s
200:	learn: 0.0629989	test: 0.0630962	best: 0.0630962 (200)	total: 3.71s	remaining: 1m 28s
300:	learn: 0.0608903	test: 0.0613346	best: 0.0613346 (300)	total: 5.55s	remaining: 1m 26s
400:	learn: 0.0599902	test: 0.0606768	best: 0.0606768 (400)	total: 7.44s	remaining: 1m 25s
500:	learn: 0.0593200	test: 0.0602854	best: 0.0602854 (500)	total: 9.43s	remaining: 1m 24s
600:	learn: 0.0587778	test: 0.0600304	best: 0.0600304 (600)	total: 11.5s	remaining: 1m 23s
700:	learn: 0.0583608	test: 0.0598586	best: 0.0598586 (700)	total: 13.3s	remaining: 1m 21s
800:	learn: 0.0580119	test: 0.0597521	best: 0.0597520 (799)	total: 15.4s	remaining: 1m 20s
900:	learn: 0.0577159	test: 0.0596893	best: 0.0596891 (899)	total: 17.2s	remaining: 1m 18s
1000:	learn: 0.0574393	test: 0.0596483	best: 0.0596483 (1000)	total: 19s	remaining: 1m 

# HP Tuning with new features

In [51]:
import itertools
import os
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Load / prepare training data
drop_cols = ['id', 'Calories']
X = train.drop(columns=drop_cols)
y = train['Calories']
cat_features = ['Sex']

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Log-transform the target for RMSLE
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

# Parameter grid
param_grid = {
    'depth': [6, 8, 10],
    'learning_rate': [0.03, 0.07, 0.1],
    'l2_leaf_reg': [3, 7],
    'iterations': [1000, 2000],
    'random_strength': [1, 5, 10],
    'bootstrap_type': ['Bayesian', 'Bernoulli'],
    'grow_policy': ['SymmetricTree', 'Depthwise']
}

# Convert grid to list of dicts (one per combination)
def expand_grid(param_grid):
    keys = list(param_grid.keys())
    values = list(param_grid.values())
    combos = list(itertools.product(*values))
    return [dict(zip(keys, combo)) for combo in combos]

all_combinations = expand_grid(param_grid)

# File to cache results
results_file = "catboost_grid_results_calorie_rmsle.csv"
if os.path.exists(results_file):
    results_df = pd.read_csv(results_file)
else:
    results_df = pd.DataFrame(columns=list(param_grid.keys()) + ["rmsle"])

# Run only for new combinations
new_results = []

for params in all_combinations:
    # Check if this combination has already been run
    mask = (results_df[list(params)] == pd.Series(params)).all(axis=1)
    if mask.any():
        print(f"✅ Skipping already-run: {params}")
        continue

    print(f"🚀 Running: {params}")
    try:
        model = CatBoostRegressor(cat_features=cat_features, verbose=0, eval_metric='RMSE', **params)
        # Train on log-transformed target
        model.fit(X_train, y_train_log)
        # Predict and compute RMSLE
        preds_log = model.predict(X_val)
        rmsle = sqrt(mean_squared_error(y_val_log, preds_log))
        print(f"→ RMSLE: {rmsle:.4f}")

        # Save result after each run
        result_row = {**params, 'rmsle': rmsle}
        results_df = pd.concat([results_df, pd.DataFrame([result_row])], ignore_index=True)
        results_df.to_csv(results_file, index=False)
        print(f"💾 Result saved to {results_file}")

    except Exception as e:
        print(f"❌ Error with params {params}: {e}")

# Append new results and save
if new_results:
    results_df = pd.concat([results_df, pd.DataFrame(new_results)], ignore_index=True)
    results_df.to_csv(results_file, index=False)
    print(f"💾 Results saved to {results_file}")

# Show best
best = results_df.sort_values('rmsle').iloc[0]
print("\n🏆 Best Config:")
print(best)

🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 1, 'bootstrap_type': 'Bayesian', 'grow_policy': 'SymmetricTree'}
→ RMSLE: 0.0603
💾 Result saved to catboost_grid_results_calorie_rmsle.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 1, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Depthwise'}


/var/folders/d3/s0jdmf113zs_4gc6g42z31fh0000gn/T/ipykernel_36927/2466759518.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([result_row])], ignore_index=True)


→ RMSLE: 0.0600
💾 Result saved to catboost_grid_results_calorie_rmsle.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 1, 'bootstrap_type': 'Bernoulli', 'grow_policy': 'SymmetricTree'}
→ RMSLE: 0.0602
💾 Result saved to catboost_grid_results_calorie_rmsle.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 1, 'bootstrap_type': 'Bernoulli', 'grow_policy': 'Depthwise'}
→ RMSLE: 0.0600
💾 Result saved to catboost_grid_results_calorie_rmsle.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 5, 'bootstrap_type': 'Bayesian', 'grow_policy': 'SymmetricTree'}
→ RMSLE: 0.0604
💾 Result saved to catboost_grid_results_calorie_rmsle.csv
🚀 Running: {'depth': 6, 'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'random_strength': 5, 'bootstrap_type': 'Bayesian', 'grow_policy': 'Depthwise'}
→ RMSLE: 0.0601
💾 Result saved to 

In [52]:
import numpy as np
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

# Define columns to exclude from features
exclude_cols = ['id', 'Calories']
target_col = 'Calories'

# Split features and target from train
X_train = train.drop(columns=exclude_cols)
y_train = train[target_col]

# Log-transform the target for RMSLE
y_train_log = np.log1p(y_train)

# Prepare test set (same feature columns)
X_test = test.drop(columns=['id'])

# Identify categorical features
categorical_features = ['Sex']

# Initialize KFold
cv = KFold(n_splits=5, random_state=42, shuffle=True)

# Initialize array to store test predictions
test_preds = np.zeros(len(test))

# K-Fold Cross-Validation
for fold, (idx_train, idx_valid) in enumerate(cv.split(X_train, y_train)):
    print(f'\nFold {fold + 1}')
    
    # Split data
    X_tr, X_val = X_train.iloc[idx_train], X_train.iloc[idx_valid]
    y_tr, y_val = y_train_log.iloc[idx_train], y_train_log.iloc[idx_valid]
    
    # Define Pools
    train_pool = Pool(X_tr, y_tr, cat_features=categorical_features)
    val_pool = Pool(X_val, y_val, cat_features=categorical_features)
    test_pool = Pool(X_test, cat_features=categorical_features)
    
    # Train the model
    model = CatBoostRegressor(
        iterations=2000,
        learning_rate=0.03,
        depth=10,
        random_strength=10,
        l2_leaf_reg=3,
        bootstrap_type='Bernoulli',
        grow_policy='SymmetricTree',
        eval_metric='RMSE',  # RMSE on log-transformed target = RMSLE
        random_seed=42,
        early_stopping_rounds=100,
        verbose=100
    )
    
    model.fit(train_pool, eval_set=val_pool)
    
    # Predict on test set and revert log-transform
    fold_preds = np.expm1(model.predict(test_pool))
    test_preds += fold_preds

# Average predictions across folds
test_preds /= cv.get_n_splits()

# Add predictions to test dataframe
test['Calories'] = test_preds

# Optional: preview results
test[['id','Calories']].to_csv('catboost_k_fold_new_hpt.csv',index=False)
print(test[['id', 'Calories']].head())


Fold 1
0:	learn: 0.9371140	test: 0.9362425	best: 0.9362425 (0)	total: 38.7ms	remaining: 1m 17s
100:	learn: 0.0954647	test: 0.0952194	best: 0.0952194 (100)	total: 1.86s	remaining: 34.9s
200:	learn: 0.0654030	test: 0.0656548	best: 0.0656548 (200)	total: 3.71s	remaining: 33.2s
300:	learn: 0.0623339	test: 0.0628706	best: 0.0628706 (300)	total: 5.6s	remaining: 31.6s
400:	learn: 0.0608693	test: 0.0616435	best: 0.0616435 (400)	total: 7.49s	remaining: 29.9s
500:	learn: 0.0597914	test: 0.0608233	best: 0.0608233 (500)	total: 9.47s	remaining: 28.3s
600:	learn: 0.0588962	test: 0.0602764	best: 0.0602764 (600)	total: 11.5s	remaining: 26.7s
700:	learn: 0.0582981	test: 0.0599942	best: 0.0599942 (700)	total: 13.5s	remaining: 25s
800:	learn: 0.0578421	test: 0.0598480	best: 0.0598480 (799)	total: 15.8s	remaining: 23.6s
900:	learn: 0.0574499	test: 0.0597301	best: 0.0597301 (900)	total: 17.8s	remaining: 21.7s
1000:	learn: 0.0570999	test: 0.0596688	best: 0.0596688 (1000)	total: 19.8s	remaining: 19.8s
1100:

In [53]:
train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,bmi,dur_body_heart,BMR
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0,22.955684,107666.0,1826.25
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0,22.582709,26996.0,1137.75
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0,24.690405,23402.4,1230.25
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0,24.414062,106837.5,2005.00
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0,22.136740,103530.0,1296.50


In [55]:
train.Duration.value_counts()

Duration
9.0     28900
26.0    28768
19.0    28433
15.0    28084
11.0    27967
25.0    27765
5.0     27493
28.0    27239
16.0    27215
8.0     26743
10.0    26723
14.0    26701
3.0     26153
21.0    26109
6.0     26102
4.0     25969
7.0     25061
12.0    24790
27.0    24758
24.0    24749
17.0    24611
13.0    24249
29.0    23856
20.0    23775
2.0     23689
18.0    23418
22.0    23208
23.0    22360
1.0     12648
30.0    12464
Name: count, dtype: int64

In [56]:
test.Duration.value_counts()

Duration
9.0     9622
19.0    9532
11.0    9434
26.0    9417
5.0     9323
15.0    9214
25.0    9152
28.0    9120
16.0    9064
8.0     8964
10.0    8948
14.0    8868
3.0     8726
21.0    8658
6.0     8648
4.0     8590
12.0    8452
7.0     8355
27.0    8341
17.0    8223
24.0    8170
13.0    8085
29.0    7902
22.0    7885
18.0    7876
20.0    7837
2.0     7833
23.0    7377
30.0    4241
1.0     4143
Name: count, dtype: int64

In [57]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import TargetEncoder
import xgboost as xgb
import numpy as np
import pandas as pd

categorical_cols = ['Sex']
cv = KFold(n_splits=5, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

# Log-transform the target
y_log = np.log1p(y)  # log(1 + y) for RMSLE-friendly training

for idx_train, idx_valid in cv.split(X, y):
    X_train, y_train = X.iloc[idx_train].copy(), y_log.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid].copy(), y_log.iloc[idx_valid]
    X_test = test[X.columns].copy()

    # Target Encoding for categorical column
    encoder = TargetEncoder(random_state=42)
    X_train['Sex'] = encoder.fit_transform(X_train[['Sex']], y_train)[:, 0]
    X_valid['Sex'] = encoder.transform(X_valid[['Sex']])[:, 0]
    X_test['Sex'] = encoder.transform(X_test[['Sex']])[:, 0]

    model = xgb.XGBRegressor(
        n_estimators=2000,
        max_depth=10,
        learning_rate=0.03,
        subsample=0.7,
        colsample_bytree=0.7,
        objective='reg:squarederror',  # L2 loss
        eval_metric='rmse',
        early_stopping_rounds=100,
        random_state=42,
        verbosity=1,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=100
    )

    y_pred += np.expm1(model.predict(X_test))

# Average predictions over folds
y_pred /= cv.get_n_splits()

# Create submission file
pred_xgb = y_pred
sample_submission = pd.read_csv('sample_submission.csv')
submission_xgb = pd.DataFrame({'id': sample_submission.id, 'Calories': pred_xgb})
submission_xgb.to_csv('submission_xgb.csv', index=False)

[0]	validation_0-rmse:0.93414
[100]	validation_0-rmse:0.07703
[200]	validation_0-rmse:0.05988
[300]	validation_0-rmse:0.05974
[386]	validation_0-rmse:0.05974
[0]	validation_0-rmse:0.93653
[100]	validation_0-rmse:0.07810
[200]	validation_0-rmse:0.06077
[300]	validation_0-rmse:0.06056
[387]	validation_0-rmse:0.06061
[0]	validation_0-rmse:0.93517
[100]	validation_0-rmse:0.07699
[200]	validation_0-rmse:0.05984
[300]	validation_0-rmse:0.05969
[376]	validation_0-rmse:0.05971
[0]	validation_0-rmse:0.93370
[100]	validation_0-rmse:0.07700
[200]	validation_0-rmse:0.06003
[300]	validation_0-rmse:0.05988
[389]	validation_0-rmse:0.05991
[0]	validation_0-rmse:0.93442
[100]	validation_0-rmse:0.07718
[200]	validation_0-rmse:0.06015
[300]	validation_0-rmse:0.06000
[376]	validation_0-rmse:0.06003


In [79]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder
import xgboost as xgb
import numpy as np
import pandas as pd

categorical_cols = ['Sex']

# Discretize 'duration' into 5 bins for stratification
duration_bins = pd.qcut(X['Duration'], q=5, labels=False, duplicates='drop')

# Use StratifiedKFold for stratified splitting based on duration bins
cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
y_pred = np.zeros(len(test))

# Log-transform the target
y_log = np.log1p(y)  # log(1 + y) for RMSLE-friendly training

for idx_train, idx_valid in cv.split(X, duration_bins):
    X_train, y_train = X.iloc[idx_train].copy(), y_log.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid].copy(), y_log.iloc[idx_valid]
    X_test = test[X.columns].copy()

    # Target Encoding for categorical column
    encoder = TargetEncoder(random_state=42)
    X_train['Sex'] = encoder.fit_transform(X_train[['Sex']], y_train)[:, 0]
    X_valid['Sex'] = encoder.transform(X_valid[['Sex']])[:, 0]
    X_test['Sex'] = encoder.transform(X_test[['Sex']])[:, 0]

    model = xgb.XGBRegressor(
        n_estimators=2000,
        max_depth=10,
        learning_rate=0.03,
        subsample=0.7,
        colsample_bytree=0.7,
        objective='reg:squarederror',  # L2 loss
        eval_metric='rmse',
        early_stopping_rounds=100,
        random_state=42,
        verbosity=1,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=100
    )

    y_pred += np.expm1(model.predict(X_test))

# Average predictions over folds
y_pred /= cv.get_n_splits()

# Create submission file
pred_xgb = y_pred
sample_submission = pd.read_csv('sample_submission.csv')
submission_xgb = pd.DataFrame({'id': sample_submission.id, 'Calories': pred_xgb})
submission_xgb.to_csv('submission_xgb.csv', index=False)

[0]	validation_0-rmse:0.93585
[100]	validation_0-rmse:0.09281
[200]	validation_0-rmse:0.06295
[300]	validation_0-rmse:0.06119
[400]	validation_0-rmse:0.06072
[500]	validation_0-rmse:0.06057
[600]	validation_0-rmse:0.06059
[615]	validation_0-rmse:0.06061
[0]	validation_0-rmse:0.93293
[100]	validation_0-rmse:0.09143
[200]	validation_0-rmse:0.06255
[300]	validation_0-rmse:0.06088
[400]	validation_0-rmse:0.06040
[500]	validation_0-rmse:0.06028
[600]	validation_0-rmse:0.06030
[653]	validation_0-rmse:0.06032
[0]	validation_0-rmse:0.93667
[100]	validation_0-rmse:0.09201
[200]	validation_0-rmse:0.06195
[300]	validation_0-rmse:0.06014
[400]	validation_0-rmse:0.05969
[500]	validation_0-rmse:0.05959
[600]	validation_0-rmse:0.05961
[629]	validation_0-rmse:0.05962
[0]	validation_0-rmse:0.93631
[100]	validation_0-rmse:0.09176
[200]	validation_0-rmse:0.06168
[300]	validation_0-rmse:0.05986
[400]	validation_0-rmse:0.05933
[500]	validation_0-rmse:0.05916
[600]	validation_0-rmse:0.05915
[652]	validation

# Hyper parameter tuning

In [80]:
import itertools
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import TargetEncoder
from sklearn.metrics import mean_squared_error
from math import sqrt
import xgboost as xgb

# Load / prepare data (assuming X, y, test, and sample_submission are already defined)
categorical_cols = ['Sex']

# Discretize 'Duration' into 5 bins for stratification
duration_bins = pd.qcut(X['Duration'], q=5, labels=False, duplicates='drop')

# Parameter grid for XGBoost
param_grid = {
    'max_depth': [6, 8, 10],
    'learning_rate': [0.03, 0.07, 0.1],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9],
    'n_estimators': [1000, 2000],
    'min_child_weight': [1, 5]
}

# Convert grid to list of dicts (one per combination)
def expand_grid(param_grid):
    keys = list(param_grid.keys())
    values = list(param_grid.values())
    combos = list(itertools.product(*values))
    return [dict(zip(keys, combo)) for combo in combos]

all_combinations = expand_grid(param_grid)

# File to cache results
results_file = "xgboost_grid_results_calorie_rmsle.csv"
if os.path.exists(results_file):
    results_df = pd.read_csv(results_file)
else:
    results_df = pd.DataFrame(columns=list(param_grid.keys()) + ["rmsle"])

# Stratified K-Fold
cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

# Log-transform the target
y_log = np.log1p(y)

# Run only for new combinations
new_results = []

for params in all_combinations:
    # Check if this combination has already been run
    mask = (results_df[list(params)] == pd.Series(params)).all(axis=1)
    if mask.any():
        print(f"✅ Skipping already-run: {params}")
        continue

    print(f"🚀 Running: {params}")
    try:
        rmsle_scores = []
        for idx_train, idx_valid in cv.split(X, duration_bins):
            X_train, y_train = X.iloc[idx_train].copy(), y_log.iloc[idx_train]
            X_valid, y_valid = X.iloc[idx_valid].copy(), y_log.iloc[idx_valid]

            # Target Encoding for categorical column
            encoder = TargetEncoder(random_state=42)
            X_train['Sex'] = encoder.fit_transform(X_train[['Sex']], y_train)[:, 0]
            X_valid['Sex'] = encoder.transform(X_valid[['Sex']])[:, 0]

            # Initialize XGBoost model
            model = xgb.XGBRegressor(
                objective='reg:squarederror',
                eval_metric='rmse',
                early_stopping_rounds=100,
                random_state=42,
                verbosity=0,
                **params
            )

            # Train model
            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                verbose=False
            )

            # Predict and compute RMSLE
            preds_log = model.predict(X_valid)
            rmsle = sqrt(mean_squared_error(y_valid, preds_log))
            rmsle_scores.append(rmsle)

        # Average RMSLE across folds
        avg_rmsle = np.mean(rmsle_scores)
        print(f"→ Average RMSLE: {avg_rmsle:.4f}")

        # Save result after each run
        result_row = {**params, 'rmsle': avg_rmsle}
        results_df = pd.concat([results_df, pd.DataFrame([result_row])], ignore_index=True)
        results_df.to_csv(results_file, index=False)
        print(f"💾 Result saved to {results_file}")

    except Exception as e:
        print(f"❌ Error with params {params}: {e}")

# Show best configuration
best = results_df.sort_values('rmsle').iloc[0]
print("\n🏆 Best Config:")
print(best)

# Optionally, train final model with best parameters and generate submission
print("\nTraining final model with best parameters...")
y_pred = np.zeros(len(test))
best_params = best.drop('rmsle').to_dict()

for idx_train, idx_valid in cv.split(X, duration_bins):
    X_train, y_train = X.iloc[idx_train].copy(), y_log.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid].copy(), y_log.iloc[idx_valid]
    X_test = test[X.columns].copy()

    # Target Encoding
    encoder = TargetEncoder(random_state=42)
    X_train['Sex'] = encoder.fit_transform(X_train[['Sex']], y_train)[:, 0]
    X_valid['Sex'] = encoder.transform(X_valid[['Sex']])[:, 0]
    X_test['Sex'] = encoder.transform(X_test[['Sex']])[:, 0]

    # Train model with best parameters
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        eval_metric='rmse',
        early_stopping_rounds=100,
        random_state=42,
        verbosity=0,
        **best_params
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=False
    )

    y_pred += np.expm1(model.predict(X_test))

# Average predictions over folds
y_pred /= cv.get_n_splits()

# Create submission file
submission_xgb = pd.DataFrame({'id': sample_submission['id'], 'Calories': y_pred})
submission_xgb.to_csv('submission_xgb_tuned.csv', index=False)
print("💾 Submission file saved as 'submission_xgb_tuned.csv'")

🚀 Running: {'max_depth': 6, 'learning_rate': 0.03, 'subsample': 0.7, 'colsample_bytree': 0.7, 'n_estimators': 1000, 'min_child_weight': 1}
→ Average RMSLE: 0.0600
💾 Result saved to xgboost_grid_results_calorie_rmsle.csv
🚀 Running: {'max_depth': 6, 'learning_rate': 0.03, 'subsample': 0.7, 'colsample_bytree': 0.7, 'n_estimators': 1000, 'min_child_weight': 5}


/var/folders/d3/s0jdmf113zs_4gc6g42z31fh0000gn/T/ipykernel_36927/1219100247.py:99: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([result_row])], ignore_index=True)


→ Average RMSLE: 0.0600
💾 Result saved to xgboost_grid_results_calorie_rmsle.csv
🚀 Running: {'max_depth': 6, 'learning_rate': 0.03, 'subsample': 0.7, 'colsample_bytree': 0.7, 'n_estimators': 2000, 'min_child_weight': 1}
→ Average RMSLE: 0.0599
💾 Result saved to xgboost_grid_results_calorie_rmsle.csv
🚀 Running: {'max_depth': 6, 'learning_rate': 0.03, 'subsample': 0.7, 'colsample_bytree': 0.7, 'n_estimators': 2000, 'min_child_weight': 5}
→ Average RMSLE: 0.0599
💾 Result saved to xgboost_grid_results_calorie_rmsle.csv
🚀 Running: {'max_depth': 6, 'learning_rate': 0.03, 'subsample': 0.7, 'colsample_bytree': 0.9, 'n_estimators': 1000, 'min_child_weight': 1}
→ Average RMSLE: 0.0602
💾 Result saved to xgboost_grid_results_calorie_rmsle.csv
🚀 Running: {'max_depth': 6, 'learning_rate': 0.03, 'subsample': 0.7, 'colsample_bytree': 0.9, 'n_estimators': 1000, 'min_child_weight': 5}
→ Average RMSLE: 0.0602
💾 Result saved to xgboost_grid_results_calorie_rmsle.csv
🚀 Running: {'max_depth': 6, 'learning_r

# ENsemble the two best cases (In log space)

In [87]:
data1=pd.read_csv('submission_xgb (1).csv')
data2=pd.read_csv('catboost_5_fold_str (1).csv')

data = data1.merge(data2, on='id', suffixes=('_xgb', '_cat'))

In [88]:
data.head()

,id,Calories_xgb,Calories_cat
0,750000,27.488697,27.206506
1,750001,107.910565,107.878140
2,750002,87.331006,87.098264
3,750003,126.202942,125.542912
4,750004,76.272044,76.180985


In [89]:
import pandas as pd
import numpy as np



# 1. Simple average of the two predictions
data['Calories_avg'] = (data['Calories_xgb'] + data['Calories_cat']) / 2

# 2. Log-scale average (RMSLE-style averaging)
data['Calories_log_avg'] = np.expm1(
    (np.log1p(data['Calories_xgb']) + np.log1p(data['Calories_cat'])) / 2
)

print(data[['id', 'Calories_xgb', 'Calories_cat', 'Calories_avg', 'Calories_log_avg']].head())

       id  Calories_xgb  Calories_cat  Calories_avg  Calories_log_avg
0  750000     27.488697     27.206506     27.347601         27.347250
1  750001    107.910565    107.878140    107.894353        107.894351
2  750002     87.331006     87.098264     87.214635         87.214558
3  750003    126.202942    125.542912    125.872927        125.872498
4  750004     76.272044     76.180985     76.226515         76.226501


In [90]:
# Save simple average with 'Calories' as the second column
data[['id', 'Calories_avg']].rename(columns={'Calories_avg': 'Calories'}).to_csv('submission_reg_avg.csv', index=False)

# Save log-scale average with 'Calories' as the second column
data[['id', 'Calories_log_avg']].rename(columns={'Calories_log_avg': 'Calories'}).to_csv('submission_log_avg.csv', index=False)

# MAy be fill with the mean value